# Project

Welcome to the group project! The project is based on the [ACM RecSys 2021 Challenge](https://recsys-twitter.com/).

- Detailed information about the task, submission and grading can be found in a [dedicates site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1217340).
- Information about the dataset structure [on this site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1218810).

In [1]:
team_name = "team_15" # your team name e.g. 'team_1'
team_members = [("",""),
                ("","")] # [("Jane Doe","012345678"), ("John Doe","012345678")]


In [134]:
print(team_name)
print(team_members)

team_15
[('', ''), ('', '')]


In [135]:
path_to_data = 'data/project/training/'
dataset_type = 'one_hour' # all_sorted, one_day, one_hour, one_week

In [136]:
try:
    import pandas as pd
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip3 install pandas
    else:
        !pip3 install --user pandas

In [137]:
import os
import re
import csv
import datetime

from model import reply_pred_model, retweet_pred_model, quote_pred_model, fav_pred_model 

all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

def parse_input_line(line):
    features = line #.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engaging_user_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    tweet_timestamp = features[all_features_to_idx['tweet_timestamp']]
    
    return tweet_id, user_id, input_feats, tweet_timestamp



def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id, user_id, features, tweet_timestamp = parse_input_line(row)                                                           
                    reply_pred = reply_pred_model(features) # reply_model
                    retweet_pred = retweet_pred_model(features) # retweet_model
                    quote_pred = quote_pred_model(features) # pred_model
                    fav_pred = fav_pred_model(features) # fav_model
                    
                    # print(str(tweet_timestamp))
#                     print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')


In [138]:
def load_data(filename):
        data = pd.read_csv(filename, sep='\x01', names=all_features, index_col=False)
        return data
    

In [139]:
# evaluate_test_set()

In [140]:
data = load_data(path_to_data + dataset_type)

# We choose first 5k rows in order to work faster with the data
data = data.head(5000)

In [141]:
data.head()

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,101\t100\t100\t100\t100\t100\t100\t100\t100\t1...,NaN,395A05A1E8A0A4CEB2E623281C7A41EE,NaN,NaN,NaN,TopLevel,313ECD3A1E5BB07406E4249475C2D6D6,1614207600,55F619B7474C1BA0C8EE03C2A31C534C,6567,4006,False,1456262411,736278C2FEC488516CDA4ED6952A2154,1139,1126,False,1601425426,False,NaN,NaN,NaN,1.614208e+09
1,101\t100\t100\t100\t100\t100\t216\t216\t10243\...,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,Photo,758E6F75A253992C7070F6B8A8A891A6,6B7D92057ACA0F97EFB5B724D3C963E4,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1614207600,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,19D5367D835484236CAF9DBEF475FF7A,82,76,False,1495813718,False,NaN,NaN,NaN,NaN
2,101\t100\t100\t100\t100\t100\t216\t216\t10243\...,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,Photo,758E6F75A253992C7070F6B8A8A891A6,6B7D92057ACA0F97EFB5B724D3C963E4,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1614207600,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,40BEB04CF8D3CB02449879668656FFDB,108,351,False,1506038593,False,NaN,NaN,NaN,NaN
3,101\t100\t100\t100\t100\t100\t216\t216\t10243\...,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,Photo,758E6F75A253992C7070F6B8A8A891A6,6B7D92057ACA0F97EFB5B724D3C963E4,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1614207600,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,6415C94D3C27BA84C069DE049EBB3EDE,69,106,False,1562533868,False,NaN,NaN,NaN,NaN
4,101\t100\t100\t100\t100\t100\t216\t216\t10243\...,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,Photo,758E6F75A253992C7070F6B8A8A891A6,6B7D92057ACA0F97EFB5B724D3C963E4,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1614207600,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,7E614D5881BC18768880CC374C4BE821,24,180,False,1302057914,False,NaN,NaN,NaN,NaN


In [156]:
def columns_to_list(data, columns):
    
    for col in columns:
        data[col] = data[col].str.split('\t')
    return data


def columns_to_timestamps(data, columns):
    for col in columns:  
        data[col] = data[col].apply(lambda x: pd.Timestamp(x, unit='s'))
        
    return data
    
cols_to_list = ['text_tokens', 'hashtags', 'present_media', 'present_links', 'present_domains']
data = columns_to_list(data, cols_to_list)    

cols_to_timestamps = ['tweet_timestamp', 'enaging_user_account_creation', 'reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']
data = columns_to_timestamps(data, cols_to_timestamps)  

In [143]:
pd.set_option('display.max_columns', None)
print(data.shape)
display(data.head(50))

(5000, 24)


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[101, 100, 100, 100, 100, 100, 100, 100, 100, ...",NaN,395A05A1E8A0A4CEB2E623281C7A41EE,NaN,NaN,NaN,TopLevel,313ECD3A1E5BB07406E4249475C2D6D6,2021-02-24 23:00:00,55F619B7474C1BA0C8EE03C2A31C534C,6567,4006,False,1456262411,736278C2FEC488516CDA4ED6952A2154,1139,1126,False,2020-09-30 00:23:46,False,NaT,NaT,NaT,2021-02-24 23:05:04
1,"[101, 100, 100, 100, 100, 100, 216, 216, 10243...",NaN,81E8247F4E74A0FCDBA911E1A3CB5412,[Photo],[758E6F75A253992C7070F6B8A8A891A6],[6B7D92057ACA0F97EFB5B724D3C963E4],TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,19D5367D835484236CAF9DBEF475FF7A,82,76,False,2017-05-26 15:48:38,False,NaT,NaT,NaT,NaT
2,"[101, 100, 100, 100, 100, 100, 216, 216, 10243...",NaN,81E8247F4E74A0FCDBA911E1A3CB5412,[Photo],[758E6F75A253992C7070F6B8A8A891A6],[6B7D92057ACA0F97EFB5B724D3C963E4],TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,40BEB04CF8D3CB02449879668656FFDB,108,351,False,2017-09-22 00:03:13,False,NaT,NaT,NaT,NaT
3,"[101, 100, 100, 100, 100, 100, 216, 216, 10243...",NaN,81E8247F4E74A0FCDBA911E1A3CB5412,[Photo],[758E6F75A253992C7070F6B8A8A891A6],[6B7D92057ACA0F97EFB5B724D3C963E4],TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,6415C94D3C27BA84C069DE049EBB3EDE,69,106,False,2019-07-07 21:11:08,False,NaT,NaT,NaT,NaT
4,"[101, 100, 100, 100, 100, 100, 216, 216, 10243...",NaN,81E8247F4E74A0FCDBA911E1A3CB5412,[Photo],[758E6F75A253992C7070F6B8A8A891A6],[6B7D92057ACA0F97EFB5B724D3C963E4],TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,7E614D5881BC18768880CC374C4BE821,24,180,False,2011-04-06 02:45:14,False,NaT,NaT,NaT,NaT
5,"[101, 100, 100, 100, 100, 100, 216, 216, 10243...",NaN,81E8247F4E74A0FCDBA911E1A3CB5412,[Photo],[758E6F75A253992C7070F6B8A8A891A6],[6B7D92057ACA0F97EFB5B724D3C963E4],TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,9CFE74AF494F669C35496B000C5843AB,269,494,False,2011-11-11 23:21:18,False,NaT,NaT,NaT,NaT
6,"[101, 100, 100, 100, 100, 100, 216, 216, 10243...",NaN,81E8247F4E74A0FCDBA911E1A3CB5412,[Photo],[758E6F75A253992C7070F6B8A8A891A6],[6B7D92057ACA0F97EFB5B724D3C963E4],TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,B5FAB4F5EC1B12BE7E7FE0C27662EB80,770,1323,False,2010-06-18 17:25:31,True,NaT,NaT,NaT,2021-02-24 23:42:41
7,"[101, 100, 100, 11045, 12589, 10545, 27920, 71...",[2C8377CDAFF61B5431417E4615B5C079],40729594A0EC3DB7D690C6789EC11992,[Video],NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,13F0E6E89E19B4E2F6C6F881E6B5B35E,4428,376,False,1454502209,BDAFE5407EF4AA3E63E12B8074CA6951,201,1322,False,2010-08-03 04:23:42,False,NaT,NaT,NaT,NaT
8,"[101, 100, 100, 216, 216, 137, 10183, 10115, 1...",NaN,E5DE595BFF4F548CE9CD32B599FAB516,[Video],NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-24 23:00:00,193A666BBC290075BB8E78672D0153A6,175968,161,False,1569911252,11612C81933A4F93C5F985AAFE3E04CB,47,356,False,2019-08-21 07:17:56,False,NaT,NaT,NaT,2021-02-24 23:07:39
9,"[101, 100, 100, 216, 216, 137, 10183, 10115, 1...",NaN,E5DE595BFF4F548CE9CD32B599FAB516,[Video],NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-24 23:00:00,193A666BBC290075BB8E78672D0153A6,

In [157]:
data.head()

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,NaN,NaN,395A05A1E8A0A4CEB2E623281C7A41EE,NaN,NaN,NaN,TopLevel,313ECD3A1E5BB07406E4249475C2D6D6,2021-02-24 23:00:00,55F619B7474C1BA0C8EE03C2A31C534C,6567,4006,False,1456262411,736278C2FEC488516CDA4ED6952A2154,1139,1126,False,2020-09-30 00:23:46,False,NaT,NaT,NaT,2021-02-24 23:05:04
1,NaN,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,NaN,NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,19D5367D835484236CAF9DBEF475FF7A,82,76,False,2017-05-26 15:48:38,False,NaT,NaT,NaT,NaT
2,NaN,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,NaN,NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,40BEB04CF8D3CB02449879668656FFDB,108,351,False,2017-09-22 00:03:13,False,NaT,NaT,NaT,NaT
3,NaN,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,NaN,NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,6415C94D3C27BA84C069DE049EBB3EDE,69,106,False,2019-07-07 21:11:08,False,NaT,NaT,NaT,NaT
4,NaN,NaN,81E8247F4E74A0FCDBA911E1A3CB5412,NaN,NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,9B49D384D56A65E00A12D8349EB46CE5,47951,681,True,1305833605,7E614D5881BC18768880CC374C4BE821,24,180,False,2011-04-06 02:45:14,False,NaT,NaT,NaT,NaT


# Splitting dataset into train and test
### Splitting the training set - one hour into train and test data. The training dataset is used for model training and the test dataset for testing the trained model

In [162]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size= 0.20)

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
1146,NaN,NaN,0397E6D2EB9382DC320FDA29E84F1B99,NaN,NaN,NaN,TopLevel,E7F038DE3EAD397AEC9193686C911677,2021-02-24 23:00:00,8410955CFC9F8F5FE3E64178F95C6A52,5556387,186813,True,1261912560,CF74DDC15A0CAF047A2C86FBEA39A3F6,214,284,False,2013-09-24 09:18:01,False,NaT,NaT,NaT,NaT
1660,NaN,NaN,8B5C4313D10F9AE9645797A7B223F179,NaN,NaN,NaN,TopLevel,E7F038DE3EAD397AEC9193686C911677,2021-02-24 23:00:00,B2872A2C72682E793EFAE8C8B6283851,148554,43,True,1313815746,16978A9CEAA53A227AC16D053DFAB72E,72,149,False,2020-06-27 13:27:15,False,NaT,NaT,NaT,NaT
24,NaN,NaN,ED7CEC4228CAE723FBEB31CDF524CA55,NaN,NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,5A38357D649427FD67C3E867AFE7AD21,49760,507,False,1585196685,CDF51BC421CF4089F67C56BC668DB177,38,108,False,2012-08-15 05:54:02,False,NaT,2021-02-24 23:00:59,NaT,2021-02-24 23:00:59
3531,NaN,NaN,82C9BE1B8A793F9B77E817E2D883EA46,NaN,NaN,NaN,TopLevel,313ECD3A1E5BB07406E4249475C2D6D6,2021-02-24 23:00:00,AB45914D28FD633B496F6B780A637238,5886,4,False,1592484694,12C266D4AD246B780701F605B0E36F90,122,250,False,2020-03-26 06:49:05,False,NaT,NaT,NaT,NaT
1715,NaN,NaN,C5D7F9A68B18861286BF2C8BB57999F8,NaN,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-24 23:00:00,8174FCBA17143D373769F87911E2D668,145830,7352,True,1233158740,0F6BD1B53186B2648E98C4FF607CF673,150,123,False,2018-08-30 03:47:45,False,NaT,NaT,NaT,NaT


In [163]:
train_data.head()

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
64,NaN,NaN,E4053CF8A9B94EEB28C7A93ED26A0F5E,NaN,NaN,NaN,Quote,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,1BC2E629F0ACD18E114E57CD0CE5B2BD,84497,765,True,1270918505,91530A0105A82CEB9615E3E08FC08AE7,52,174,False,2019-01-26 00:24:27,False,NaT,NaT,NaT,NaT
962,NaN,NaN,0397E6D2EB9382DC320FDA29E84F1B99,NaN,NaN,NaN,TopLevel,E7F038DE3EAD397AEC9193686C911677,2021-02-24 23:00:00,8410955CFC9F8F5FE3E64178F95C6A52,5556387,186813,True,1261912560,9CDABEB8B4A97A2DF50ADBE7144E5DB6,85,258,False,2015-08-10 15:55:25,False,NaT,NaT,NaT,NaT
96,NaN,NaN,2B94B6A6B87EC5F21539D6B5F438F6B5,NaN,NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,DBFDF1319E8DA74985B8C4FB963FB8D6,14620,1360,False,1593798882,6D77A8858E2CE27D2BB1A6A9F94F0CE3,1900,1090,False,2009-05-04 20:56:53,True,NaT,NaT,NaT,NaT
3462,NaN,NaN,4F3B0BD2F19BAA67634F2E391F2863B0,NaN,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-24 23:00:00,8802C28382252D9D8707AEC8773B3717,32685,1398,False,1463146228,315A7A5F6717660718F1AB84A2928F4F,19850,34,False,2019-05-02 03:42:55,False,NaT,NaT,NaT,NaT
823,NaN,NaN,0397E6D2EB9382DC320FDA29E84F1B99,NaN,NaN,NaN,TopLevel,E7F038DE3EAD397AEC9193686C911677,2021-02-24 23:00:00,8410955CFC9F8F5FE3E64178F95C6A52,5556387,186813,True,1261912560,6F88874B09523B2705D5A83771996BEA,97,504,False,2018-08-15 16:38:16,False,NaT,NaT,NaT,NaT


In [164]:
test_data.head()

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
1146,NaN,NaN,0397E6D2EB9382DC320FDA29E84F1B99,NaN,NaN,NaN,TopLevel,E7F038DE3EAD397AEC9193686C911677,2021-02-24 23:00:00,8410955CFC9F8F5FE3E64178F95C6A52,5556387,186813,True,1261912560,CF74DDC15A0CAF047A2C86FBEA39A3F6,214,284,False,2013-09-24 09:18:01,False,NaT,NaT,NaT,NaT
1660,NaN,NaN,8B5C4313D10F9AE9645797A7B223F179,NaN,NaN,NaN,TopLevel,E7F038DE3EAD397AEC9193686C911677,2021-02-24 23:00:00,B2872A2C72682E793EFAE8C8B6283851,148554,43,True,1313815746,16978A9CEAA53A227AC16D053DFAB72E,72,149,False,2020-06-27 13:27:15,False,NaT,NaT,NaT,NaT
24,NaN,NaN,ED7CEC4228CAE723FBEB31CDF524CA55,NaN,NaN,NaN,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,2021-02-24 23:00:00,5A38357D649427FD67C3E867AFE7AD21,49760,507,False,1585196685,CDF51BC421CF4089F67C56BC668DB177,38,108,False,2012-08-15 05:54:02,False,NaT,2021-02-24 23:00:59,NaT,2021-02-24 23:00:59
3531,NaN,NaN,82C9BE1B8A793F9B77E817E2D883EA46,NaN,NaN,NaN,TopLevel,313ECD3A1E5BB07406E4249475C2D6D6,2021-02-24 23:00:00,AB45914D28FD633B496F6B780A637238,5886,4,False,1592484694,12C266D4AD246B780701F605B0E36F90,122,250,False,2020-03-26 06:49:05,False,NaT,NaT,NaT,NaT
1715,NaN,NaN,C5D7F9A68B18861286BF2C8BB57999F8,NaN,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-24 23:00:00,8174FCBA17143D373769F87911E2D668,145830,7352,True,1233158740,0F6BD1B53186B2648E98C4FF607CF673,150,123,False,2018-08-30 03:47:45,False,NaT,NaT,NaT,NaT


# Evaluation

# Create a Ratings Matrix

# Extract Social Network

# Implement a Baseline

In [145]:
# hidden


In [146]:
# feel free to edit

In [147]:
# feel free to edit

In [148]:
# feel free to edit

In [149]:
# feel free to edit

In [150]:
# feel free to edit

In [151]:
# feel free to edit

In [152]:
# feel free to edit

In [153]:
# feel free to edit

In [154]:
# feel free to edit

In [155]:
# feel free to edit